In [32]:
from bs4 import BeautifulSoup
from tqdm import tqdm
import requests
import os

In [28]:
website_base = 'https://bulbapedia.bulbagarden.net'

# Make a request to the website
r = requests.get('https://bulbapedia.bulbagarden.net/wiki/List_of_Pok%E9mon_by_National_Pok%E9dex_number')
r_html = r.text

# Create a BeautifulSoup object and specify the parser
soup = BeautifulSoup(r_html, 'html.parser')


elements = soup.find_all(class_='roundy')
print(len(elements))
all_links = []
for tr in elements:
    sub_elements = tr.find_all('img')

    for img in sub_elements:
        #print(img.parent.get('href'))
        all_links.append(website_base+img.parent.get('href'))


distinct_links = list(set(all_links))
print(distinct_links[:5])
print(len(distinct_links))


10
['https://bulbapedia.bulbagarden.net/wiki/Rhyperior_(Pok%C3%A9mon)', 'https://bulbapedia.bulbagarden.net/wiki/Turtwig_(Pok%C3%A9mon)', 'https://bulbapedia.bulbagarden.net/wiki/Meloetta_(Pok%C3%A9mon)', 'https://bulbapedia.bulbagarden.net/wiki/Cinderace_(Pok%C3%A9mon)', 'https://bulbapedia.bulbagarden.net/wiki/Zamazenta_(Pok%C3%A9mon)']
1025


In [30]:
all_img_links = []
for link in tqdm(distinct_links):
    r = requests.get(link)
    r_html = r.text
    soup = BeautifulSoup(r_html, 'html.parser')
    tables = soup.find_all(class_='roundy', attrs={'style': 'background:#FFF;', 'width': '100%'})
    for table in tables:
        elements = table.find_all('a', attrs={'class': 'extiw'})
        for a in elements:
            title = a.get('title')
            if title and title.startswith('a:Category:'):
                all_img_links.append(a.get('href'))
                break
        break

print(len(all_img_links))
print(all_img_links[:5])

100%|██████████| 1025/1025 [06:53<00:00,  2.48it/s]

1025
['https://archives.bulbagarden.net/wiki/Category:Rhyperior', 'https://archives.bulbagarden.net/wiki/Category:Turtwig', 'https://archives.bulbagarden.net/wiki/Category:Meloetta', 'https://archives.bulbagarden.net/wiki/Category:Cinderace', 'https://archives.bulbagarden.net/wiki/Category:Zamazenta']


In [44]:
imgs_to_retrieve = []
img_filter = ['_dream', '_anime']

for link in tqdm(all_img_links):
    r = requests.get(link)
    r_html = r.text
    soup = BeautifulSoup(r_html, 'html.parser')
    gallery = soup.find(class_='gallery')
    elements = gallery.find_all('li')
    counter = 0
    for li in elements:
        gallerytext = li.find('div', attrs={'class': 'gallerytext'}).find('a')
        title = gallerytext.get('title') 
        if title and 'spr ' not in title.lower() and (counter < 11 or any(substring in title.lower() for substring in img_filter)):
            imgs_to_retrieve.append(website_base+gallerytext.get('href'))
        counter += 1

print(len(imgs_to_retrieve))
print(imgs_to_retrieve[:25])

100%|██████████| 1025/1025 [09:55<00:00,  1.72it/s]

9765
['https://bulbapedia.bulbagarden.net/wiki/File:0464Rhyperior.png', 'https://bulbapedia.bulbagarden.net/wiki/File:464Rhyperior_DP_anime.png', 'https://bulbapedia.bulbagarden.net/wiki/File:464Rhyperior_XY_anime.png', 'https://bulbapedia.bulbagarden.net/wiki/File:464OD.png', 'https://bulbapedia.bulbagarden.net/wiki/File:Ani464OD.png', 'https://bulbapedia.bulbagarden.net/wiki/File:464Rhyperior_Dream.png', 'https://bulbapedia.bulbagarden.net/wiki/File:0387Turtwig.png', 'https://bulbapedia.bulbagarden.net/wiki/File:387Turtwig_BDSP.png', 'https://bulbapedia.bulbagarden.net/wiki/File:387Turtwig_Pt.png', 'https://bulbapedia.bulbagarden.net/wiki/File:387Turtwig_Dream_2.png', 'https://bulbapedia.bulbagarden.net/wiki/File:387Turtwig_Dream_3.png', 'https://bulbapedia.bulbagarden.net/wiki/File:387Turtwig_WF.png', 'https://bulbapedia.bulbagarden.net/wiki/File:387Turtwig_DP_anime_2.png', 'https://bulbapedia.bulbagarden.net/wiki/File:387Turtwig_DP_anime.png', 'https://bulbapedia.bulbagarden.net/wi

In [62]:
extra_filters = ['_Contest_']

no_od = [link for link in imgs_to_retrieve if 'OD.' not in link and 'ODS.' not in link]
no_od = list(set(no_od))

filtered = [link for link in no_od if all(substring not in link for substring in extra_filters)]

In [86]:
from joblib import Parallel, delayed
import numpy as np
#image_links = imgs_to_retrieve
#image_links = no_od
image_links = filtered

folder = 'dataset/bulbapedia'
amount_jobs = 10

#print(os.path.basename(image_links[0]).split(':')[1])

missing_links = [link for link in tqdm(image_links) if not os.path.exists(os.path.join(folder, os.path.basename(link).split(':')[1]))]
print(len(missing_links))

image_links = missing_links

def img_downloader(image_links, pbar):
    #for link in tqdm(image_links):
    for link in image_links:
        if os.path.exists(os.path.join(folder, os.path.basename(link).split(':')[1])):
            continue
        r = requests.get(link)
        r_html = r.text
        soup = BeautifulSoup(r_html, 'html.parser')
        elements = soup.find(class_='fullImageLink')
        a = elements.find('a')
        img_link = a.get('href')

        img_data = requests.get(img_link).content

        # Get the image name from the link
        img_name = os.path.basename(img_link)

        os.makedirs(folder, exist_ok=True)

        # Write the image data to a file in the specified folder
        with open(os.path.join(folder, img_name), 'wb') as f:
            f.write(img_data)

        
        pbar.update()

    #break

np.random.shuffle(image_links)
splits = [list(split) for split in np.array_split(image_links, amount_jobs)]



with tqdm(total=len(image_links)) as pbar:
    Parallel(n_jobs=amount_jobs, prefer="threads", verbose=0)(delayed(img_downloader)(split, pbar) for split in splits)


  0%|          | 30/8512 [00:00<01:45, 80.03it/s] 

100%|██████████| 8512/8512 [00:12<00:00, 661.93it/s] 


4


100%|██████████| 4/4 [00:03<00:00,  1.06it/s]


In [103]:
from PIL import Image
from io import BytesIO
import shutil

bah_folder = 'dataset/bulbapedia_not_good'
amount_jobs = 20

def is_white_or_transparent(img):
    pixel = img.getpixel((0, 0))
    #print(type(pixel))
    return pixel == (255, 255, 255, 255) or pixel[3] == 0

def filter_files(files, pbar):
    for filename in files:
        if filename.endswith('.png') or filename.endswith('.jpg'):  # Add or modify as needed
            img_path = os.path.join(folder, filename)
            #print(img_path)
            img = Image.open(img_path).convert('RGBA')

            # Check if the pixel at (0, 0) is not white or transparent
            os.makedirs(bah_folder, exist_ok=True)
            if not is_white_or_transparent(img):
                # Delete the image
                bah_path = os.path.join(bah_folder, filename)
                shutil.move(img_path, bah_path)
            
        pbar.update()


file_list = os.listdir(folder)
np.random.shuffle(file_list)
splits = [list(split) for split in np.array_split(file_list, amount_jobs)]



with tqdm(total=len(file_list)) as pbar:
    Parallel(n_jobs=amount_jobs, prefer="threads", verbose=0)(delayed(filter_files)(split, pbar) for split in splits)

100%|██████████| 7164/7164 [00:11<00:00, 609.78it/s]
